In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
import scipy.interpolate

In [2]:
# Set up a function for the comoving distance

def com(z_, OmC, OmB, HH0, Nnu):
    """ Gets the comoving distance in units of Mpc/h at a given redshift, z_ (assuming the cosmology defined in the params file). """
    
    OmR	=	2.47*10**(-5)/(HH0/100.)**2
    OmN	=	Nnu*(7./8.)*(4./11.)**(4./3.)*OmR
    OmL = 1. - OmC - OmB - OmR - OmN
    c=2.99792458*10**(8)
    H0	=	10**(5)/c
    
    def chi_int(z):
        return 1. / (H0 * ( (OmC+OmB)*(1+z)**3 + OmL + (OmR+OmN) * (1+z)**4 )**(0.5))

    if hasattr(z_, "__len__"):
        chi=np.zeros((len(z_)))
        for zi in range(0,len(z_)):
            #print "zi in com=", zi
            chi[zi] = scipy.integrate.quad(chi_int,0,z_[zi])[0]
    else:
        chi = scipy.integrate.quad(chi_int, 0, z_)[0]

    return chi

In [3]:
# Define a function to get redshift distribution (in terms of specz)
def get_NofZ(dNdzpar, dNdztype, z):
    """ Returns the dNdz of the sources."""

    if (dNdztype == 'Nakajima'):
        # dNdz takes form like in Nakajima et al. 2011 equation 3
        a = dNdzpar[0]
        zs = dNdzpar[1]
        nofz_ = (z / zs)**(a-1) * np.exp( -0.5 * (z / zs)**2)	
    elif (dNdztype == 'Smail'):
        # dNdz take form like in Smail et al. 1994
        alpha = dNdzpar[0]
        z0 = dNdzpar[1]
        beta = dNdzpar[2]
        nofz_ = z**alpha * np.exp( - (z / z0)**beta)
    else:
        print "dNdz type "+str(dNdztype)+" not yet supported; exiting."
        exit()
        
    norm = scipy.integrate.simps(nofz_, z)

    return nofz_ / norm

In [4]:
# Set up the Schechter luminosity function parameters from Krause et al. 2015

z_1 = 1.37# 
# Define a few absolute magnitude limits for which we will check this:
#mlim_2 = 27 # LSST shapes
mlim_1 = 25

#z = scipy.linspace(0.02, 5.00, 500)

#alpha_h = 0.081; beta_h = 2.1
Mr_s = -20.34; Q = 1.8; alpha = -0.57
phi_0 = 1.1 * 10**(-2); P = -1.2

phi_s_1 = phi_0 * 10**(0.4 * P*z_1)
#phi_s_2 = phi_0 * 10**(0.4 * P*z_2)
Ms_1 = Mr_s - Q * ( z_1 - 0.1 )
#Ms_2 = Mr_s - Q * ( z_2 - 0.1)
Ls_1 = 10.**(-0.4 *Ms_1)
#Ls_2 = 10.**(-0.4 *Ms_2)
print "Ms_1=", Ms_1, "Ls_1=", Ls_1
#print "Ms_2=", Ms_2, "Ls_2=", Ls_2

Ms_1= -22.626 Ls_1= 1123052347.54


In [5]:
# Import the k correction and EC correction from Poggianti (assuming elliptical galaxies)
# Only data up to z=3 so have just extended at a constant the highest-z value
(z_k, kcorr, x,x,x) = np.loadtxt('../txtfiles/kcorr.dat', unpack=True)
(z_e, ecorr, x,x,x) = np.loadtxt('../txtfiles/ecorr.dat', unpack=True)

kcorr_interp = scipy.interpolate.interp1d(z_k, kcorr)
ecorr_interp = scipy.interpolate.interp1d(z_e, ecorr)

kcorr_1 = kcorr_interp(z_1)
ecorr_1 = ecorr_interp(z_1)
#kcorr_2 = kcorr_interp(z_2)
#ecorr_2 = ecorr_interp(z_2)

#print "kcorr_1=", kcorr_1, "kcorr_2=", kcorr_2
#print "ecorr_1=", ecorr_1, "ecorr_2=", ecorr_2

In [6]:
# Get the absolute magnitude that corresponds to the limiting apparent magnitude
dl_1 = com(z_1, 0.2, 0.05, 70., 3.046) * (1. + z_1)
#dl_2 = com(z_2, 0.2, 0.05, 70., 3.046) * (1. + z_2)
#print "dl 1=", dl_1, "dl 2=", dl_2

#Mlim_1 = -22.6
Mlim_1 = mlim_1 - (5. * np.log10(dl_1) + 25. + kcorr_1 + ecorr_1)
Llim_1 = 10.**(-0.4 * Mlim_1)
print "Mlim 1=", Mlim_1, "Llim 1=", Llim_1
#Mlim_2 = mlim_2 - (5. * np.log10(dl_2) + 25. + kcorr_2 + ecorr_2)
#Llim_2 = 10.**(-0.4 * Mlim_2)
#print "Mlim 2=", Mlim_2, "Llim 2=", Llim_2

Mlim 1= -20.1082036079 Llim 1= 110479434.643


In [7]:
L0 = 10.**(-0.4 * (-22.))
print "L0=", L0

#L_list = [0]*len(z)
#phi_func = np.zeros((len(z), 1000))
#phi_func = np.zeros(1000)
#for i in range(0,len(z)):
#for j in range(0,1000):
#print "i=", i, "j=", j
#L = scipy.logspace(np.log10(Llim[i]), 10., 1000)
L_1 = scipy.logspace(np.log10(Llim_1), np.log10(4.*10**9), 1000)
#L_2 = scipy.logspace(np.log10(Llim_2), np.log10(4.*10**9), 1000)

#L_list[i] = L
#phi_func[i,j] = phi_s[i] * (L[j] / Ls[i]) ** alpha * np.exp(- L[j] / Ls[i])
phi_func_1= phi_s_1* (L_1 / Ls_1) ** alpha * np.exp(- L_1 / Ls_1)
#phi_func_2= phi_s_2* (L_2 / Ls_2) ** alpha * np.exp(- L_2 / Ls_2)

norm_1 = scipy.integrate.simps(phi_func_1 * L_1, np.log(L_1))
#norm_2 = scipy.integrate.simps(phi_func_2 * L_2, np.log(L_2))
print "norm 1=", norm_1  #, "norm 2=", norm_2

# Check <L>/L0
avg_L = scipy.integrate.simps(phi_func_1 * L_1 * (L_1 / L0) ,np.log(L_1) ) / scipy.integrate.simps(phi_func_1 * L_1, np.log(L_1))
print "avg_L=", avg_L

#plt.figure(figsize=(10,10))
#plt.semilogx(L_1, phi_func_1/norm_1, 'm', label='SDSS')
#plt.hold(True)
#plt.xlabel('Luminosity (arbitrary units)', fontsize=20)
#plt.ylabel('$\phi$', fontsize=20)
#plt.tick_params(axis='both', labelsize='18')
#plt.title('Luminosity function', fontsize=20)
#plt.semilogx(L_2, phi_func_2/norm_2, 'g', label='LSST+DESI')
#plt.savefig('../plots/Luminosity_functions.pdf')
#plt.legend()
#plt.show()
        

L0= 630957344.48
norm 1= 3302170.31281
avg_L= 1.18334585964


In [ ]:


#a_h = [0]*(len(z))
#for i in range(0, len(z)):
#num = scipy.integrate.simps(phi_func[i, :] * alpha_h * (L_list[i] / L0)**beta_h, L_list[i])
a_h_1 = scipy.integrate.simps(phi_func_1 / norm_1 * alpha_h * (L_1 / L0)**beta_h * L_1, np.log(L_1))
#a_h_2 = scipy.integrate.simps(phi_func_2 / norm_2 * alpha_h * (L_2 / L0)**beta_h * L_2, np.log(L_2))

plt.figure(figsize=(10,10))
plt.semilogx(L_1, phi_func_1 / norm_1* alpha_h * (L_1 / L0)**beta_h, 'm', label='SDSS')
#plt.hold(True)
#plt.semilogx(L_2, phi_func_2 / norm_2* alpha_h * (L_2 / L0)**beta_h, 'g', label='LSST+DESI')
plt.xlabel("Luminosity (arbitrary units)", fontsize=20)
plt.ylabel("$a_h(L) \phi(L)$", fontsize=20)
plt.tick_params(axis='both', labelsize='18')
plt.savefig('../plots/Lum_func_times_ah.pdf')
plt.legend()
plt.show()

print "ah 1=", a_h_1 #, "ah 2=", a_h_2

In [ ]:
# Get the normalized redshift distribution in terms of spec z
#dNdz = get_NofZ([1.24, 0.51, 1.01], 'Smail', z)
#dNdz = get_NofZ([2.338, 0.303], 'Nakajima', z)

# and integrate over it

#a_h_intz = scipy.integrate.simps(a_h*dNdz, z)
print "A=", a_h